# Explore your architecture

// tag::metricsTop10LoCAndCC[]
.The top 10 types with the highest LoC (lines of code) and aggregated CC (cyclomatic complexity).

In [1]:
This demonstrates the difference between graph and table

SyntaxError: invalid syntax (1258449722.py, line 1)

In [2]:
[source, cypher]
----
MATCH
  (:Artifact)-[:CONTAINS]->(type:Type)-[:DECLARES]->(method:Method)
RETURN
  type AS Type, sum(method.cyclomaticComplexity) AS CC, sum(method.effectiveLineCount) AS LoC
ORDER BY
  LoC DESC, CC DESC
LIMIT
  10
----

SyntaxError: invalid syntax (1182437714.py, line 2)

But obviously we need a more structure view here to gather some helpful informations.

In [3]:
[source,cypher]
.The top 10 types with the highest LoC (lines of code) and CC (cyclomatic complexity).
----
MATCH
  (:Artifact)-[:CONTAINS]->(type:Type)-[:DECLARES]->(method:Method)
RETURN
  type.name,sum(method.cyclomaticComplexity) AS CC, sum(method.effectiveLineCount) AS LoC
ORDER BY
  LoC DESC, CC DESC
LIMIT
  10
----

SyntaxError: invalid syntax (4267869118.py, line 2)

## Modules:

As we are talking about architecture, we might be interested in creating a big picture of the architecture.

So let's start getting information about the modules of the application/system.

Let's try to figure out the technologie framework and the modules within the application:

In [ ]:
MATCH
  (:Main:Artifact)-[:CONTAINS]->(root:Package)-[:CONTAINS]->(module:Package)
WHERE
  root.fqn CONTAINS "spring"
SET
  module:Module
RETURN
  module.name as Module, module.fqn as Package
ORDER BY
  module.name

As a result we can see the different technologies used within the starter.spring.io

Now that we have the different modules in the project, we might be interested in the dependencies between the modules:

In [ ]:
[source,cypher]
-----
MATCH
  (module1:Module)-[:CONTAINS*]->(t1:Type),
  (module2:Module)-[:CONTAINS*]->(t2:Type),
  (t1)-[:DEPENDS_ON]->(t2)
WHERE
  module1 <> module2
WITH
  module1, module2, COUNT(*) AS weight
MERGE
  (module1)-[dependsOn:DEPENDS_ON_MODULE]->(module2)
SET
  dependsOn.weight = weight
RETURN
  module1, dependsOn, module2

Let us assume that there is even more information required and we need to drill down this dependency graph from top to types:

In [ ]:
[source,cypher]
----
MATCH
  (module:Module)-[:CONTAINS*]->(type:Type)
OPTIONAL MATCH
  (type)-[dependsOn:DEPENDS_ON]->(:Type)<-[:CONTAINS*]-(:Module)
RETURN
  {
    role: "graph",
    parent: module,
    nodes: collect(type),
    relationships: collect(dependsOn)
  }
----

We see there a point when the information is out reach as we can not dive deep into such a graph, at least it is not human readable without effort.

Of course, some will say "You have to structure your graph correctly" and at the hand they might be right, because if we have a look at the class-diagram, like this:

In [ ]:
[source,cypher]
----
MATCH
  (module:Module:Package)-[:CONTAINS*]->(type:Type)
OPTIONAL MATCH
  (type)-[extends:EXTENDS|IMPLEMENTS]->(:Type)
OPTIONAL MATCH
  (type)-[:DECLARES]->(field:Field)
OPTIONAL MATCH
  (type)-[:DECLARES]->(method:Method)
RETURN
  *
----

We gather a structured information about what is in the project and how it depends on each other. But still for me the question remains: Is there a more usable view we can get with jqassistant? 

And, no surprise, there is one (or multiple) solution(s) to archieve this goal.

Let's have a look at the same query again, but with some additional configuration:

In [5]:
// tag::moduleClassDiagram[]
[[module:ClassDiagram]]
[source,cypher,role=concept,requiresConcepts="module:Module",reportType="plantuml-class-diagram"]
----
MATCH
  (module:Module:Package)-[:CONTAINS*]->(type:Type)
OPTIONAL MATCH
  (type)-[extends:EXTENDS|IMPLEMENTS]->(:Type)
OPTIONAL MATCH
  (type)-[:DECLARES]->(field:Field)
OPTIONAL MATCH
  (type)-[:DECLARES]->(method:Method)
RETURN
  *
----

SyntaxError: invalid syntax (1766096814.py, line 1)

## Hands on architecture review

Let's switch to our IDE and try to get more useful information:

In [ ]:
:toc: left
= Architecture Analyses: Rules, Concept, Constraints, Reports and Diagrams

// tag::default[]
[[default]]
[role=group,includesGroups="module:Default,metrics:Default,package:Default"]
== jQAssistant Rules

This section describes that default rules that are executed during each build.
// end::default[]

//include::maven.adoc[]
include::metrics.adoc[]
include::module.adoc[]
include::package.adoc[]

// tag::summary[]
== jQAssistant Summary

include::jQA:Summary[concepts="*",constraints="*",importedConstraints="spring*:*"]
// end::summary[]


In [ ]:
// tag::metricsDefault[]
[[metrics:Default]]
[role=group,includesConcepts="metrics:*"]
== Metrics

This section describes metrics of the project.
// end::metricsDefault[]

// tag::metricsTop10LoCAndCC[]
[[metrics:Top10LoCAndCC]]
[source,cypher,role=concept]
.The top 10 types with the highest LoC (lines of code) and aggregated CC (cyclomatic complexity).
----
MATCH
  (:Artifact)-[:CONTAINS]->(type:Type)-[:DECLARES]->(method:Method)
RETURN
  type AS Type, sum(method.cyclomaticComplexity) AS CC, sum(method.effectiveLineCount) AS LoC
ORDER BY
  LoC DESC, CC DESC
LIMIT
  10
----
// end::metricsTop10LoCAndCC[]

// tag::metricsLoCTop10LoCAndCCAsCSV[]
[[metrics:Top10LoCAndCCAsCSV]]
[source,cypher,role=concept,reportType="csv"]
.The top 10 types with the highest LoC (lines of code) and CC (cyclomatic complexity).
----
MATCH
  (:Artifact)-[:CONTAINS]->(type:Type)-[:DECLARES]->(method:Method)
RETURN
  type.name,sum(method.cyclomaticComplexity) AS CC, sum(method.effectiveLineCount) AS LoC
ORDER BY
  LoC DESC, CC DESC
LIMIT
  10
----
// end::metricsLoCTop10LoCAndCCAsCSV[]


== Add exceptions to the rule

And add some exceptions from this rule:

* Methods equals() and hashCode() are generated by an IDE and need not to be tested
* For some reason, we don’t want measure test coverage for the UI package
* The `StringTool.doSomethingwithStrings` method should also be excluded
* And we know that there are 10 other violations that we want to skip for now + (but we swear to handle this Technical Debt in the next spring)

== Query to add exceptions:
[[metrics:Exceptions]]
[source,cypher]
----
MATCH (tcr:TestCoverageRange)

WITH tcr.min AS mincomplexity, tcr.max AS maxcomplexity, tcr.coverage AS coveragethreshold
MATCH (cl:Jacoco:Class)--(m:Jacoco:Method)--(c:Jacoco:Counter {type: 'COMPLEXITY'})
WHERE c.missed + c.covered >= mincomplexity AND c.missed + c.covered <= maxcomplexity
AND NOT m.signature IN ['boolean equals(java.lang.Object)', 'int hashCode()']
AND NOT(cl.fqn STARTS WITH 'de.kontext_e.demo.ui')
AND NOT(cl.fqn = 'de.kontext_e.demo.tools.StringTool'
AND m.signature = 'java.lang.String doSomethingwithStrings(java.lang.String)')

WITH m AS method, cl.fqn AS fqn, m.signature AS signature, c.missed+c.covered AS complexity, coveragethreshold AS coveragethreshold
MATCH (m)--(branches:Jacoco:Counter {type: 'BRANCH'})
WHERE m=method AND branches.covered*100/(branches.covered+branches.missed) < coveragethreshold
RETURN complexity, coveragethreshold, branches.covered*100/(branches.covered+branches.missed) AS coverage, fqn, signature
ORDER BY complexity, coverage
SKIP 10
----

== Special case: Frequently changed classes

Maybe it is also a good idea to have a *higher test coverage for frequently changed classes*.
Using the https://github.com/kontext-e/jqassistant-plugins/tree/master/git[Git Plugin by Kontext E], there is a way to test this:

[[metrics:Frequently]]
[source,cypher]
----
MATCH (c:Git:Commit)-[:CONTAINS_CHANGE]->(change:Git:Change)-[:MODIFIES]->(f:Git:File)
WHERE f.relativePath=~'.*.java'
AND NOT f.relativePath CONTAINS 'ui'
WITH count(c) AS cnt, replace(f.relativePath, '/','.') AS gitfqn
ORDER BY cnt DESC
LIMIT 10
MATCH (class:Java:Class)
WHERE gitfqn CONTAINS class.fqn
WITH cnt, class.fqn AS classfqn
MATCH (cl:Jacoco:Class)--(m:Jacoco:Method)--(c:Jacoco:Counter {type: 'COMPLEXITY'})
WHERE classfqn=cl.fqn
AND c.missed+c.covered > 3
AND NOT(m.signature ='boolean equals(java.lang.Object)')
AND NOT(m.signature ='int hashCode()')
WITH m AS method, cl.fqn AS fqn, m.signature AS signature, c.missed+c.covered AS complexity
MATCH (m)--(branches:Jacoco:Counter {type: 'BRANCH'})
WHERE m=method
AND branches.covered*100/(branches.covered+branches.missed) < 90
RETURN DISTINCT fqn, signature, complexity, branches.covered*100/(branches.covered+branches.missed) AS coverage
ORDER BY fqn
SKIP 3
----

For the 10 most often changed Java files (except the ones in the UI package), the test coverage for branches should not be lower than 90 percent for methods with more than 3 branches - with three unnamed exceptions from this rule.

== Encapsulation: Label types with internal FQNs as Internal
:fqn_internal: pass:a['<span value-key="fqn_internal">.internal.</span>']

++++
<input style="display:inline;width:30%;"  class="form-control" value=".internal." size="40">
++++

[[metrics:FQNsLabels]]
[source,cypher]
----
MATCH (t:Type) WHERE t.fqn CONTAINS {fqn_internal}
SET t:Internal
----

== API/SPI types must not extend/implement internal types
[[metrics:TypesMustNotExposeInternalTypes]]
[source,cypher]
----
MATCH (class:Class)-[:EXTENDS|IMPLEMENTS]->(supertype:Type:Internal)
WHERE NOT class:Internal
RETURN DISTINCT class as extendsInternal
----

== API/SPI methods must not expose internal types
[[metrics:MethodsMustNotExposeInternalTypes]]
[source,cypher]
----
// return values
MATCH (class:Type)-[:DECLARES]->(method:Method)
WHERE NOT class:Internal
AND method.visibility IN ["public","protected"]
AND (exists ((method)-[:RETURNS]->(:Type:Internal)) OR
    exists ((method)-[:`HAS`]->(:Parameter)-[:OF_TYPE]->(:Internal)))
RETURN method
----

== API/SPI fields must not expose internal types
[[metrics:FieldsMustNotExposeInternalTypes]]
[source,cypher]
----
MATCH (class:Class:Internal)-[:DECLARES]->(field)-[:OF_TYPE]->(fieldtype:Type:Internal)
WHERE field.visibility IN ["public","protected"]
RETURN class as internalClass, field, fieldtype as internalType
----



In [ ]:
// tag::moduleDefault[]
[[module:Default]]
[role=group,includesConcepts="module:*"]
== Module

This section describes rules regarding modules and their dependencies.
// end::moduleDefault[]

// tag::moduleModule[]
[[module:Module]]
[source,cypher,role=concept]
.All packages in the root package of the main artifact are labeled as `Module`.
----
MATCH
  (:Main:Artifact)-[:CONTAINS]->(root:Package)-[:CONTAINS]->(module:Package)
WHERE
  root.fqn CONTAINS "spring"
SET
  module:Module
RETURN
  module.name as Module, module.fqn as Package
ORDER BY
  module.name
----
// end::moduleModule[]

// tag::moduleDependencies[]
[[module:Dependencies]]
[source,cypher,role=concept,requiresConcepts="module:Module",reportType="plantuml-component-diagram"]
.A module depends on another module if there is a dependency of Java types between both.
----
MATCH
  (module1:Module)-[:CONTAINS*]->(t1:Type),
  (module2:Module)-[:CONTAINS*]->(t2:Type),
  (t1)-[:DEPENDS_ON]->(t2)
WHERE
  module1 <> module2
WITH
  module1, module2, COUNT(*) AS weight
MERGE
  (module1)-[dependsOn:DEPENDS_ON_MODULE]->(module2)
SET
  dependsOn.weight = weight
RETURN
  module1, dependsOn, module2
----
// end::moduleDependencies[]

// tag::moduleDependenciesGraphML[]
[[module:DependenciesGraphML]]
[source,cypher,role=concept,requiresConcepts="module:Dependencies",reportType="graphml"]
.Modules and their dependencies as GraphML report.
----
MATCH
  (module:Module)
OPTIONAL MATCH
  (module)-[dependsOn:DEPENDS_ON_MODULE]->(:Module)
RETURN
  *
----
// end::moduleDependenciesGraphML[]

// tag::moduleDependenciesGraphMLDrilldown[]
[[module:DependenciesGraphMLDrilldown]]
[source,cypher,role=concept,requiresConcepts="module:Module",reportType="plantuml-"]
.Modules and their dependencies as GraphML report with drill-down to type level.
----
MATCH
  (module:Module)-[:CONTAINS*]->(type:Type)
OPTIONAL MATCH
  (type)-[dependsOn:DEPENDS_ON]->(:Type)<-[:CONTAINS*]-(:Module)
RETURN
  {
    role: "graph",
    parent: module,
    nodes: collect(type),
    relationships: collect(dependsOn)
  }
----
// end::moduleDependenciesGraphMLDrilldown[]

// tag::moduleClassDiagram[]
[[module:ClassDiagram]]
[source,cypher,role=concept,requiresConcepts="module:Module",reportType="plantuml-class-diagram"]
.Class Diagram
----
MATCH
  (module:Module:Package)-[:CONTAINS*]->(type:Type)
WHERE
  module.fqn CONTAINS "io.spring.start.site"
OPTIONAL MATCH
  (type)-[extends:EXTENDS|IMPLEMENTS]->(:Type)
OPTIONAL MATCH
  (type)-[:DECLARES]->(field:Field)
OPTIONAL MATCH
  (type)-[:DECLARES]->(method:Method)
RETURN
  *
----
// end::moduleClassDiagram[]

// tag::moduleSequenceDiagram[]
[[module:SequenceDiagram]]
[source,cypher,role=concept,reportType="plantuml-sequence-diagram"]
.Sequence Diagram
----
MATCH
  (t:Type)-[d:DECLARES]->(method:Method),
  sequence=((method)-[:INVOKES*]->(:Method))
WHERE
  t.fqn CONTAINS "spring"
RETURN
  sequence
LIMIT
  50
----
// end::moduleSequenceDiagram[]



In [ ]:
[[package:Default]]
[role=group,includesConstraints="package:*"]
=== Package Structure

To improve maintainability in the long term the following rules for structuring the application into packages must be applied.

[[package:ServicePackage]]
[source,cypher,role=concept,reportType="plantuml-component-diagram"]
----
MATCH
    (root:Package),
    (a:Package),
    (b:Package),
    (c:Package),
    (root)-[:CONTAINS]->(a),
    (root)-[:CONTAINS]->(b),
    (root)-[:CONTAINS]->(c)
WHERE
    root.fqn CONTAINS "spring"
SET
    root:Root
SET
    a:Module
SET
    b:Module
SET
    c:Module
MERGE
    (b)-[d1:DEFINES_DEPENDENCY]->(a)
MERGE
    (c)-[d2:DEFINES_DEPENDENCY]->(a)
RETURN
    root, a, b, c, d1, d2
LIMIT
    100
----

//[[package:ServicePackage]]
//[source,cypher,role=constraint requiresConcepts="spring-component:Service",reportType="plantuml-package-diagram"]
//.Spring Services must be located in packages named `service`.
//----
//MATCH
//  (package:Package)-[:CONTAINS]->(service:Spring:Service)
//WHERE
//  package.name <> "service"
//RETURN
//  service as ServicesInInvalidPackage
//----



## Take a look behind the scene

When we take a look at the architecture, there is more than then the big picture to view. Now we want to take a look behind the scenes and see what's happening in the application:

Therefore we introduce a new type of diagram, called "sequence diagram"

In [ ]:
[[module:SequenceDiagram]]
[source,cypher,role=concept,reportType="plantuml-sequence-diagram"]
.Sequence Diagram
----
MATCH
  (t:Type)-[d:DECLARES]->(method:Method),
  sequence=((method)-[:INVOKES*]->(:Method))
WHERE
  t.fqn CONTAINS "spring"
RETURN
  sequence
LIMIT
  50
----

## Diagrams to work with

Let's now have a closer look to what we can do with the diagrams generated:

In [ ]:
@startuml
skinparam componentStyle uml2
!pragma layout smetana
  [io.spring.start.site.test.myModule] <<Directory File Java Package>> as n1111
  [io.spring.start.site.extension.dependency.vaadin] <<Directory File Java Package>> as n3392
  [io.spring.start.site.extension.dependency.springrestdocs] <<Directory File Java Package>> as n4224
  [io.spring.start.site.support] <<Directory File Java Package>> as n4803
  [io.spring.start] <<Directory File Java Package>> as n2628
  [io.spring.start.site] <<Directory File Java Package>> as n2629
  [io.spring.start.site.extension] <<Directory File Java Package>> as n2630
  [io.spring.start.site.extension.code] <<Directory File Java Package>> as n2631
  [io.spring.start.site.extension.code.kotlin] <<Directory File Java Package>> as n2632
  [io.spring.start.site.support.implicit] <<Directory File Java Package>> as n4813
  [io.spring.start.site.extension.build] <<Directory File Java Package>> as n4493
  [io.spring.start.site.extension.build.maven] <<Directory File Java Package>> as n4494
  [io.spring.start.site.extension.dependency.observability] <<Directory File Java Package>> as n3921
  [io.spring.start.site.extension.dependency.springsecurity] <<Directory File Java Package>> as n3986
  [io.spring.start.site.extension.dependency.thymeleaf] <<Directory File Java Package>> as n3987
  [io.spring.start.site.extension.dependency.springcloud] <<Directory File Java Package>> as n2836
  [io.spring.start.site.web] <<Directory File Java Package>> as n4692
  [io.spring.start.site.extension.dependency.reactor] <<Directory File Java Package>> as n4439
  [io.spring.start.site.extension.dependency.graphql] <<Directory File Java Package>> as n4120
  [io.spring.start.site.extension.dependency.flyway] <<Directory File Java Package>> as n4440
  [io.spring.start.site.extension.description] <<Directory File Java Package>> as n4441
  [io.spring.start.site.extension.dependency.okta] <<Directory File Java Package>> as n4121
  [io.spring.start.site.extension.dependency.testcontainers] <<Directory File Java Package>> as n4122
  [io.spring.start.site.extension.dependency.springnative] <<Directory File Java Package>> as n3486
  [io.spring.start.site.project] <<Directory File Java Package>> as n4704
  [io.spring.start.site.project.dependency] <<Directory File Java Package>> as n4705
  [io.spring.start.site.project.dependency.springcloud] <<Directory File Java Package>> as n4706
  [io.spring.start.site.extension.dependency] <<Directory File Java Package>> as n2787
  [io.spring.start.site.extension.dependency.springkafka] <<Directory File Java Package>> as n2788
  [io.spring.start.site.extension.dependency.solace] <<Directory File Java Package>> as n4078
  [io.spring.start.site.extension.dependency.springbatch] <<Directory File Java Package>> as n2803
  [io.spring.start.site.extension.dependency.liquibase] <<Directory File Java Package>> as n4343
  [io.spring.start.site.extension.dependency.springintegration] <<Directory File Java Package>> as n4344
  [io.spring.start.site.extension.dependency.springdata] <<Directory File Java Package>> as n4345
  [io.spring.start.site.extension.dependency.springamqp] <<Directory File Java Package>> as n3897
  [io.spring.start.site.extension.build.gradle] <<Directory File Java Package>> as n4605
  [io.spring.start.site.extension.dependency.springsession] <<Directory File Java Package>> as n3390
  [io.spring.start.site.extension.dependency.lombok] <<Directory File Java Package>> as n3391

n1111 --> n3392 : future implementation
n2628 --> n4344 : depends on module (1)
n2787 --> n4345 : depends on module (2)
n2630 --> n4345 : depends on module (2)
n2629 --> n4345 : depends on module (2)
n2628 --> n4345 : depends on module (2)
n2787 --> n4439 : depends on module (1)
n2630 --> n4439 : depends on module (1)
n2629 --> n4439 : depends on module (1)
n2628 --> n4439 : depends on module (1)
n2787 --> n4440 : depends on module (1)
n2630 --> n4440 : depends on module (1)
n2629 --> n4440 : depends on module (1)
n2628 --> n4440 : depends on module (1)
n2630 --> n4441 : depends on module (2)
n2629 --> n4441 : depends on module (2)
n2628 --> n4441 : depends on module (2)
n4494 --> n4493 : depends on module (2)
n4605 --> n4493 : depends on module (2)
n4493 --> n4494 : depends on module (2)
n2630 --> n4494 : depends on module (2)
n2629 --> n4494 : depends on module (2)
n2628 --> n4494 : depends on module (2)
n4493 --> n4605 : depends on module (2)
n2630 --> n4605 : depends on module (2)
n2629 --> n4605 : depends on module (2)
n2628 --> n4605 : depends on module (2)
n2629 --> n4692 : depends on module (1)
n2628 --> n4692 : depends on module (1)
n2629 --> n4704 : depends on module (4)
n2628 --> n4704 : depends on module (4)
n4704 --> n4705 : depends on module (1)
n2629 --> n4705 : depends on module (1)
n2628 --> n4705 : depends on module (1)
n4704 --> n4706 : depends on module (1)
n2629 --> n4706 : depends on module (1)
n2628 --> n4706 : depends on module (1)
n2629 --> n4803 : depends on module (12)
n2628 --> n4803 : depends on module (12)
n4122 --> n4803 : depends on module (5)
n2787 --> n4803 : depends on module (10)
n2630 --> n4803 : depends on module (10)
n4344 --> n4803 : depends on module (5)
n4813 --> n4803 : depends on module (7)
n4122 --> n4813 : depends on module (5)
n2787 --> n4813 : depends on module (10)
n2630 --> n4813 : depends on module (10)
n2629 --> n4813 : depends on module (17)
n2628 --> n4813 : depends on module (17)
n4344 --> n4813 : depends on module (5)
n4803 --> n4813 : depends on module (7)
n2629 --> n2628 : depends on module (14)
n4704 --> n2628 : depends on module (3)
n4122 --> n2628 : depends on module (6)
n2787 --> n2628 : depends on module (22)
n2630 --> n2628 : depends on module (10)
n4344 --> n2628 : depends on module (6)
n4813 --> n2628 : depends on module (7)
n2632 --> n2628 : depends on module (4)
n4494 --> n2628 : depends on module (2)
n4605 --> n2628 : depends on module (2)
n4441 --> n2628 : depends on module (2)
n4078 --> n2628 : depends on module (1)
n3392 --> n2628 : depends on module (2)
n3921 --> n2628 : depends on module (3)
n4345 --> n2628 : depends on module (2)
n2836 --> n2628 : depends on module (13)
n4224 --> n2628 : depends on module (3)
n3486 --> n2628 : depends on module (11)
n3897 --> n2628 : depends on module (1)
n2628 --> n2629 : depends on module (14)
n4704 --> n2629 : depends on module (3)
n4122 --> n2629 : depends on module (6)
n2787 --> n2629 : depends on module (22)
n2630 --> n2629 : depends on module (10)
n4344 --> n2629 : depends on module (6)
n4813 --> n2629 : depends on module (7)
n2632 --> n2629 : depends on module (4)
n4494 --> n2629 : depends on module (2)
n4605 --> n2629 : depends on module (2)
n4441 --> n2629 : depends on module (2)
n4078 --> n2629 : depends on module (1)
n3392 --> n2629 : depends on module (2)
n3921 --> n2629 : depends on module (3)
n4345 --> n2629 : depends on module (2)
n2836 --> n2629 : depends on module (13)
n4224 --> n2629 : depends on module (3)
n3486 --> n2629 : depends on module (11)
n3897 --> n2629 : depends on module (1)
n2632 --> n2630 : depends on module (4)
n4494 --> n2630 : depends on module (2)
n4605 --> n2630 : depends on module (2)
n4441 --> n2630 : depends on module (2)
n2787 --> n2630 : depends on module (12)
n4078 --> n2630 : depends on module (1)
n3392 --> n2630 : depends on module (2)
n3921 --> n2630 : depends on module (3)
n4345 --> n2630 : depends on module (2)
n2836 --> n2630 : depends on module (13)
n4344 --> n2630 : depends on module (1)
n4122 --> n2630 : depends on module (1)
n4224 --> n2630 : depends on module (3)
n3486 --> n2630 : depends on module (11)
n3897 --> n2630 : depends on module (1)
n2632 --> n2631 : depends on module (4)
n2631 --> n2632 : depends on module (4)
n2630 --> n2632 : depends on module (4)
n2629 --> n2632 : depends on module (4)
n2628 --> n2632 : depends on module (4)
n2630 --> n2787 : depends on module (12)
n2629 --> n2787 : depends on module (12)
n2628 --> n2787 : depends on module (12)
n4078 --> n2787 : depends on module (1)
n3392 --> n2787 : depends on module (2)
n3921 --> n2787 : depends on module (3)
n4345 --> n2787 : depends on module (2)
n2836 --> n2787 : depends on module (13)
n4344 --> n2787 : depends on module (1)
n4122 --> n2787 : depends on module (1)
n4224 --> n2787 : depends on module (3)
n3486 --> n2787 : depends on module (11)
n3897 --> n2787 : depends on module (1)
n2787 --> n2788 : depends on module (1)
n2630 --> n2788 : depends on module (1)
n2629 --> n2788 : depends on module (1)
n2628 --> n2788 : depends on module (1)
n2787 --> n2803 : depends on module (1)
n2630 --> n2803 : depends on module (1)
n2629 --> n2803 : depends on module (1)
n2628 --> n2803 : depends on module (1)
n2787 --> n2836 : depends on module (13)
n2630 --> n2836 : depends on module (13)
n2629 --> n2836 : depends on module (13)
n2628 --> n2836 : depends on module (13)
n2787 --> n3390 : depends on module (1)
n2630 --> n3390 : depends on module (1)
n2629 --> n3390 : depends on module (1)
n2628 --> n3390 : depends on module (1)
n2787 --> n3391 : depends on module (1)
n2630 --> n3391 : depends on module (1)
n2629 --> n3391 : depends on module (1)
n2628 --> n3391 : depends on module (1)
n2787 --> n3392 : depends on module (2)
n2630 --> n3392 : depends on module (2)
n2629 --> n3392 : depends on module (2)
n2628 --> n3392 : depends on module (2)
n2787 --> n3486 : depends on module (11)
n2630 --> n3486 : depends on module (11)
n2629 --> n3486 : depends on module (11)
n2628 --> n3486 : depends on module (11)
n2787 --> n3897 : depends on module (1)
n2630 --> n3897 : depends on module (1)
n2629 --> n3897 : depends on module (1)
n2628 --> n3897 : depends on module (1)
n2787 --> n3921 : depends on module (3)
n2630 --> n3921 : depends on module (3)
n2629 --> n3921 : depends on module (3)
n2628 --> n3921 : depends on module (3)
n2787 --> n3986 : depends on module (2)
n2630 --> n3986 : depends on module (2)
n2629 --> n3986 : depends on module (2)
n2628 --> n3986 : depends on module (2)
n2787 --> n3987 : depends on module (1)
n2630 --> n3987 : depends on module (1)
n2629 --> n3987 : depends on module (1)
n2628 --> n3987 : depends on module (1)
n2787 --> n4078 : depends on module (1)
n2630 --> n4078 : depends on module (1)
n2629 --> n4078 : depends on module (1)
n2628 --> n4078 : depends on module (1)
n2787 --> n4120 : depends on module (1)
n2630 --> n4120 : depends on module (1)
n2629 --> n4120 : depends on module (1)
n2628 --> n4120 : depends on module (1)
n2787 --> n4121 : depends on module (1)
n2630 --> n4121 : depends on module (1)
n2629 --> n4121 : depends on module (1)
n2628 --> n4121 : depends on module (1)
n2787 --> n4122 : depends on module (1)
n2630 --> n4122 : depends on module (1)
n2629 --> n4122 : depends on module (1)
n2628 --> n4122 : depends on module (1)
n2787 --> n4224 : depends on module (3)
n2630 --> n4224 : depends on module (3)
n2629 --> n4224 : depends on module (3)
n2628 --> n4224 : depends on module (3)
n2787 --> n4343 : depends on module (1)
n2630 --> n4343 : depends on module (1)
n2629 --> n4343 : depends on module (1)
n2628 --> n4343 : depends on module (1)
n2787 --> n4344 : depends on module (1)
n2630 --> n4344 : depends on module (1)
n2629 --> n4344 : depends on module (1)

@enduml


## Architecture documentation

At this point I want to make a short break and recover what we have until now:
- Rules to enrich the graph
- Concepts to gather more architecture information
- Constraints to ensure the architecture
- Rendering information in a uml diagram

At this point I was wondering if it was possible to render the diagrams directly within the documentation?

So I took a closer look at the Ascii-File of arc42 (https://arc42.org/download) and of course to the plugins provided by jqassistant and found a solution within no time as we already on the necessary stack to do the job:

- PlantUml plugin of jqassistant is able to render the diagrams directly into an ascii doc
- 